In [ ]:
import requests
from bs4 import BeautifulSoup as soup
from requests_html import AsyncHTMLSession
import pandas as pd

In [ ]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}


In [ ]:
html = requests.get('https://www.digikey.com/en/products/filter/embedded-microprocessors/694?s=N4IgTCBcDaIMYFMA2CBOB7AdiAugXyA',headers=header)


In [ ]:
html.status_code


In [ ]:
cookies = html.cookies

In [ ]:
bsobj = soup(html.content, "html.parser")

In [ ]:
next = bsobj.find(title='Next Page')
next

In [ ]:
productlist = bsobj.findAll('tr',{'class':'MuiTableRow-root'})
productlist[2].text
links = []
for i in range(2,len(productlist)):
    links.append(productlist[i].find_all('a')[1].get('href'))
links

In [ ]:
asession = AsyncHTMLSession()
r = await asession.get('https://www.digikey.com')
for f in links:
    r = await asession.get('https://www.digikey.com' + f)
    print('https://www.digikey.com' + f)

In [ ]:
df = pd.DataFrame(columns=['Product Id','In Stock','Rating','Shipping Time','Shipping Location'])

In [ ]:
for f in links:
    
    
    temp = {}
    r = await asession.get('https://www.mouser.com' + f)
    await r.html.arender()
    if r.html.find('div[ref_page_event="Link to Supplier Shipper"]'):
        h1 = r.html.find('tbody.MuiTableBody-root')
        temp['Product Id'] = h1[0].text.split('\n')[1]
        stock = r.html.find('div[data-testid = price-and-procure-title]')
        temp['In Stock'] = stock[0].text


        if (stock[0].text[0] != '0'):
            rating_count = r.html.find('div[ref_page_event="Link to Supplier Shipper"]')
            shipping = rating_count[0].text.replace('Will ship in approximately ','').split('from')
            temp['Shipping Time'] = shipping[0].strip()
            temp['Shipping Location'] = shipping[1].strip()
        else:
            temp['Shipping Time'] = 'err: no items in stock'
            temp['Shipping Location'] = 'err: no items in stock'
        print(temp)
        df = df.append(temp, ignore_index=True)


In [ ]:
display(df)